<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex07-Alpha_Vantage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory, Fundamental Analysis, and Data Driven Analytics**, Bates, Boyer, and Fletcher

# Examples Chapter 7: The Alpha Vantage API
In this example we illustrate how to load in data using the Alpha vantage API. We then plot the closing price for Coca-Cola (ticker:'KO').

### Imports and Setup
We first import the `requests` library which is one of the most widely used libraries for making HTTP requests to get data. We then set some printing options when displaying dataframes to the terminal.

In [ ]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)   # Show all columns without truncation
pd.set_option('display.width', 1000)   # Set the display width so output stays on one line
pd.set_option("display.max_rows", 20) # Force truncation to at most 20 rows (10 from top, 10 from bottom)


### Load in Data Using the API
You will first need to get your own API key to run this block of code at this link: https://www.alphavantage.co/support/#api-key.  Once you have the key, insert it in place of `your_key` in the code below. We then format the data and display a few rows.   

In [ ]:
# Replace the apikey with your own
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=KO&apikey=your_key'
r = requests.get(url)
data = r.json()

# The time series data lives under "Monthly Adjusted Time Series"
ts_data = data["Monthly Adjusted Time Series"]

# Convert to DataFrame
df = pd.DataFrame.from_dict(ts_data, orient="index")

# Rename columns to something cleaner
df.columns = [
    "Open", "High", "Low", "Close",
    "Adjusted Close", "Volume", "Dividend Amount"
]

# Convert index to datetime and sort chronologically
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# Convert string values to numeric
df = df.astype(float)
print(df)

### Plot Closing Price for Coca-Cola

In [ ]:
import matplotlib.pyplot as plt

# Plot monthly close price
plt.figure(figsize=(12,6))
plt.plot(df.index, df["Close"], color="black", linewidth=1)

plt.xlabel("Date", fontsize=14)
plt.ylabel("Closing Price (USD)", fontsize=14)
plt.title("KO Monthly Closing Price", fontsize=16)
plt.grid(False)  # cleaner look
plt.show()